In [1]:
#! pip3 install virtualenv
!pip3 install --user ipykernel


[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: python3.10 -m pip install --upgrade pip


In [2]:
! python -m ipykernel install --user --name=mychurn

/System/Library/Frameworks/Python.framework/Versions/2.7/Resources/Python.app/Contents/MacOS/Python: No module named ipykernel


In [3]:
#! pip install seaborn
! pip install tensorflow


[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: python3.10 -m pip install --upgrade pip


In [4]:
! pip install imblearn


[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: python3.10 -m pip install --upgrade pip


In [5]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from imblearn.under_sampling import TomekLinks
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import accuracy_score, roc_auc_score
from collections import Counter
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
import joblib 
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout , BatchNormalization
import tensorflow as tf
from sklearn.neural_network import MLPClassifier
from sklearn.decomposition import PCA
from sklearn.utils import shuffle

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

import json

SEED = 42
NAME = 'ai-small-preproc'
np.random.seed(SEED)

2022-12-31 03:11:42.393467: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
df = pd.read_csv("./data/bank_data_train.csv")
test = pd.read_csv("./data/bank_data_test.csv")
print(f'Размерность данных:\n\ttrain: {df.shape} \n\ttest: {test.shape}')
df.head()

Размерность данных:
	train: (355190, 116) 
	test: (88798, 116)


,ID,CR_PROD_CNT_IL,AMOUNT_RUB_CLO_PRC,PRC_ACCEPTS_A_EMAIL_LINK,APP_REGISTR_RGN_CODE,PRC_ACCEPTS_A_POS,PRC_ACCEPTS_A_TK,TURNOVER_DYNAMIC_IL_1M,CNT_TRAN_AUT_TENDENCY1M,SUM_TRAN_AUT_TENDENCY1M,...,REST_DYNAMIC_CC_3M,MED_DEBT_PRC_YWZ,LDEAL_ACT_DAYS_PCT_TR3,LDEAL_ACT_DAYS_PCT_AAVG,LDEAL_DELINQ_PER_MAXYWZ,TURNOVER_DYNAMIC_CC_3M,LDEAL_ACT_DAYS_PCT_TR,LDEAL_ACT_DAYS_PCT_TR4,LDEAL_ACT_DAYS_PCT_CURR,TARGET
0,146841,0,0.000000,NaN,NaN,NaN,NaN,0.0,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0
1,146842,0,0.041033,NaN,NaN,NaN,NaN,0.0,0.166667,0.186107,...,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0
2,146843,0,0.006915,0.0,NaN,0.0,0.0,0.0,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0
3,146844,0,0.000000,NaN,NaN,NaN,NaN,0.0,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0
4,146845,0,0.000000,NaN,NaN,NaN,NaN,0.0,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0


In [7]:
df['TARGET'].value_counts()

0    326265
1     28925
Name: TARGET, dtype: int64

In [8]:
df.columns

Index(['ID', 'CR_PROD_CNT_IL', 'AMOUNT_RUB_CLO_PRC',
       'PRC_ACCEPTS_A_EMAIL_LINK', 'APP_REGISTR_RGN_CODE', 'PRC_ACCEPTS_A_POS',
       'PRC_ACCEPTS_A_TK', 'TURNOVER_DYNAMIC_IL_1M', 'CNT_TRAN_AUT_TENDENCY1M',
       'SUM_TRAN_AUT_TENDENCY1M',
       ...
       'REST_DYNAMIC_CC_3M', 'MED_DEBT_PRC_YWZ', 'LDEAL_ACT_DAYS_PCT_TR3',
       'LDEAL_ACT_DAYS_PCT_AAVG', 'LDEAL_DELINQ_PER_MAXYWZ',
       'TURNOVER_DYNAMIC_CC_3M', 'LDEAL_ACT_DAYS_PCT_TR',
       'LDEAL_ACT_DAYS_PCT_TR4', 'LDEAL_ACT_DAYS_PCT_CURR', 'TARGET'],
      dtype='object', length=116)

## Обработка

#### Исключаем стоблцы с пустышками и преобразуем категориальные признаки

In [9]:
desc = df.describe()
desc

,ID,CR_PROD_CNT_IL,AMOUNT_RUB_CLO_PRC,PRC_ACCEPTS_A_EMAIL_LINK,APP_REGISTR_RGN_CODE,PRC_ACCEPTS_A_POS,PRC_ACCEPTS_A_TK,TURNOVER_DYNAMIC_IL_1M,CNT_TRAN_AUT_TENDENCY1M,SUM_TRAN_AUT_TENDENCY1M,...,REST_DYNAMIC_CC_3M,MED_DEBT_PRC_YWZ,LDEAL_ACT_DAYS_PCT_TR3,LDEAL_ACT_DAYS_PCT_AAVG,LDEAL_DELINQ_PER_MAXYWZ,TURNOVER_DYNAMIC_CC_3M,LDEAL_ACT_DAYS_PCT_TR,LDEAL_ACT_DAYS_PCT_TR4,LDEAL_ACT_DAYS_PCT_CURR,TARGET
count,355190.000000,355190.000000,316867.000000,155163.0,60550.000000,155163.0,155163.0,355190.000000,77112.000000,77112.000000,...,355190.000000,95713.000000,93448.000000,98175.000000,95713.000000,355190.000000,93448.000000,93448.000000,93448.000000,355190.000000
mean,368794.674875,0.105225,0.044045,0.0,50.947498,0.0,0.0,0.001305,0.416896,0.414572,...,0.007309,0.055074,0.025707,0.049943,0.009252,0.004309,0.013938,0.013938,0.013938,0.081435
std,128148.804566,0.431372,0.108449,0.0,21.777855,0.0,0.0,0.029118,0.316493,0.338612,...,0.066681,0.215909,0.115732,0.185830,0.092789,0.059852,0.097099,0.097099,0.097099,0.273503
min,146841.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.006944,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,257846.250000,0.000000,0.000000,0.0,33.000000,0.0,0.0,0.000000,0.166667,0.139645,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,368778.500000,0.000000,0.000000,0.0,54.000000,0.0,0.0,0.000000,0.300000,0.285714,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,479737.750000,0.000000,0.036608,0.0,72.000000,0.0,0.0,0.000000,0.571429,0.661195,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,590828.000000,11.000000,1.000000,0.0,89.000000,0.0,0.0,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [10]:
desc.T[desc.T['mean'] == 0]

,count,mean,std,min,25%,50%,75%,max
PRC_ACCEPTS_A_EMAIL_LINK,155163.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
PRC_ACCEPTS_A_POS,155163.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
PRC_ACCEPTS_A_TK,155163.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
PRC_ACCEPTS_A_AMOBILE,155163.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
PRC_ACCEPTS_TK,155163.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
PRC_ACCEPTS_A_MTP,155163.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CNT_ACCEPTS_TK,155163.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
PRC_ACCEPTS_A_ATM,155163.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
PRC_ACCEPTS_MTP,155163.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CNT_ACCEPTS_MTP,155163.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [63]:
null_cols = list(desc.T[desc.T['mean'] == 0].index)

10

In [12]:
def preprocessing(data):
    df = data.copy()
    meta = {}
    
    df.drop(null_cols, axis=1, inplace=True)
    cat_feat = df.select_dtypes(include=['object']).columns.tolist()
    for col in cat_feat:
        meta[col] = LabelEncoder()
        df[col] = meta[col].fit_transform(df[col].fillna('0'))
    df.fillna(0, inplace=True)
    
    for col in df.drop('TARGET', axis=1).columns:
        mean_, std_ = df[col].mean(), df[col].std()
        df[col] = (df[col] - mean_) / std_
    return df.drop(['ID', 'TARGET'], axis=1), df.TARGET, (df.columns, meta)

In [13]:
df_preproc, y, meta = preprocessing(df)
print(meta)

(Index(['ID', 'CR_PROD_CNT_IL', 'AMOUNT_RUB_CLO_PRC', 'APP_REGISTR_RGN_CODE',
       'TURNOVER_DYNAMIC_IL_1M', 'CNT_TRAN_AUT_TENDENCY1M',
       'SUM_TRAN_AUT_TENDENCY1M', 'AMOUNT_RUB_SUP_PRC',
       'SUM_TRAN_AUT_TENDENCY3M', 'CLNT_TRUST_RELATION',
       ...
       'REST_DYNAMIC_CC_3M', 'MED_DEBT_PRC_YWZ', 'LDEAL_ACT_DAYS_PCT_TR3',
       'LDEAL_ACT_DAYS_PCT_AAVG', 'LDEAL_DELINQ_PER_MAXYWZ',
       'TURNOVER_DYNAMIC_CC_3M', 'LDEAL_ACT_DAYS_PCT_TR',
       'LDEAL_ACT_DAYS_PCT_TR4', 'LDEAL_ACT_DAYS_PCT_CURR', 'TARGET'],
      dtype='object', length=106), {'CLNT_TRUST_RELATION': LabelEncoder(), 'APP_MARITAL_STATUS': LabelEncoder(), 'APP_KIND_OF_PROP_HABITATION': LabelEncoder(), 'CLNT_JOB_POSITION_TYPE': LabelEncoder(), 'CLNT_JOB_POSITION': LabelEncoder(), 'APP_DRIVING_LICENSE': LabelEncoder(), 'APP_EDUCATION': LabelEncoder(), 'APP_TRAVEL_PASS': LabelEncoder(), 'APP_CAR': LabelEncoder(), 'APP_POSITION_TYPE': LabelEncoder(), 'APP_EMP_TYPE': LabelEncoder(), 'APP_COMP_TYPE': LabelEncoder()

### Разделяем выборку на тестовую и тренировочную ( 20% )

In [71]:
%%time
data = {}
data['X_train'], data['X_test'], data['y_train'], data['y_test'] = train_test_split(df_preproc, y, test_size=0.2)

CPU times: user 206 ms, sys: 16.1 ms, total: 222 ms
Wall time: 221 ms


In [70]:
# %%time
# tl = TomekLinks()
# data['X_train'], data['y_train'] = tl.fit_resample(data['X_train'], data['y_train'])
# for name, df_ in data.items():
#     print(f'{name}: {df_ .shape}')

## Обучение моделей

### Вспомогательные функции для сбора данных

In [15]:
def myScore(data, prediction, name, config):
    result = {
        'name': name,
        'train AUC': roc_auc_score(data['y_train'], prediction[0]),
        'test AUC': roc_auc_score(data['y_test'], prediction[1]),
        'train ACC': accuracy_score(data['y_train'], (prediction[0] > 0.5).astype(int)),
        'test ACC': accuracy_score(data['y_test'], (prediction[1] > 0.5).astype(int)),
        'config': config
    }
    
    for name, val in result.items():
        print(f'{name}: {val}')
        
    return result

In [16]:
def myPredict(res_table, data, model_class, config):
    
    model = model_class(config)
    if config.get('model'):
        del config['model']
    model_name = model_class.__name__ + ': ' + json.dumps(config)
    
    model.fit(data['X_train'], data['y_train']),
    models[model_name] = model
    prediction = (
        model.predict(data['X_train']),
        model.predict(data['X_test'])
    )
    
    res_table = res_table.append(myScore(data, prediction, model_name, config), ignore_index=True)
    
    return res_table, models 

In [17]:
models = {}
res_table = pd.DataFrame(columns=['name', 'train AUC', 'test AUC', 'train ACC', 'test ACC', 'config'])
res_table

,name,train AUC,test AUC,train ACC,test ACC,config


### Обертки для моделей

#### BernoulliNB

In [18]:
class WrapperNaive:
    def __init__(self, hp):
        self.binarize = hp.get('binarize', 5)
        
    def fit(self, X, y):
        X = X.values.astype('float32')

        self.model = BernoulliNB(binarize = self.binarize)
        self.model.fit(X, y)
        
    def predict(self, X):
        return self.model.predict_proba(X)[:, 1]

#### Keras

In [19]:
class WrapperKeras:
    def __init__(self, hp):
        self.epochs = hp.get('epochs', 50)
        self.batch_size = hp.get('batch_size', 512)
        self.model = hp['model']
        
    def fit(self, X, y):
        X = X.values.astype('float32')
        y = pd.get_dummies(y).values.astype('float32')

        self.model.fit(X, y, epochs=self.epochs, batch_size=self.batch_size)
        
    def predict(self, X):
        return self.model.predict(X.values.astype('float32'))[:, 1]

#### GridSearch RandomForestClassifier

In [20]:
class WrapperRFGridSearch:
    def __init__(self, param_grid, metric = 'roc_auc'):
        self.param_grid = param_grid
        self.metric = metric
    
    def fit(self, X, y):
        gs = GridSearchCV(
            estimator=RandomForestClassifier(), 
            param_grid=self.param_grid, 
            scoring=self.metric,
            cv=5,
            verbose=1,
            n_jobs=-1
        )
        
        gs.fit(X, y)
        self.model = gs.best_estimator_

    def predict(self, X):
        return self.model.predict_proba(X)[:, 1]

#### GridSearch MLPClassifier

In [21]:
class WrapperMLPGrid:
    def __init__(self, param_grid, metric = 'roc_auc'):
        self.param_grid = param_grid
        self.metric = metric
    
    def fit(self, X, y):
        gs = GridSearchCV(
            estimator=MLPClassifier(), 
            param_grid=self.param_grid, 
            cv=5,
            verbose=1,
            n_jobs=-1,
            scoring=self.metric
        )
        gs.fit(X, y)
        
        self.model = gs.best_estimator_

    def predict(self, X):
        return self.model.predict_proba(X)[:, 1]

### Конструируем модель для keras 

In [68]:
def createKerasModel(inputN):
    model = Sequential()
    model.add(Dense(40, input_shape=(inputN, ), activation='tanh'))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    model.add(Dense(45, activation='tanh'))
    model.add(BatchNormalization())
    model.add(Dense(65, activation='tanh'))
    model.add(BatchNormalization())
    model.add(Dense(45, activation='tanh'))
    model.add(BatchNormalization())
    model.add(Dense(45, activation='tanh'))
    model.add(BatchNormalization())
    model.add(Dense(2, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam')

    return model

### Запускаем модели

#### Baseline naive classifier

In [23]:
res_table, models = myPredict(res_table, data, WrapperNaive, {'binarize': 2})

name: WrapperNaive: {"binarize": 2}
train AUC: 0.6127456030359923
test AUC: 0.6141469579105038
train ACC: 0.891287057631127
test ACC: 0.8922970804358231
config: {'binarize': 2}


In [24]:
res_table, models = myPredict(res_table, data, WrapperNaive, {'binarize': 5})

name: WrapperNaive: {"binarize": 5}
train AUC: 0.5439329481653913
test AUC: 0.5386919968920905
train ACC: 0.9125468059348518
test ACC: 0.9136096173878769
config: {'binarize': 5}


In [25]:
res_table

,name,train AUC,test AUC,train ACC,test ACC,config
0,"WrapperNaive: {""binarize"": 2}",0.612746,0.614147,0.891287,0.892297,{'binarize': 2}
1,"WrapperNaive: {""binarize"": 5}",0.543933,0.538692,0.912547,0.913610,{'binarize': 5}


#### Запускаем Keras

* на 50 эпох

In [26]:
kerasModel = createKerasModel(data['X_train'].shape[1])
config = {'epochs': 50, 'model': kerasModel}

2022-12-31 03:12:00.007251: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [27]:
res_table, models = myPredict(res_table, data, WrapperKeras, config)

Epoch 1/50
555/555 [==============================] - 4s 3ms/step - loss: 0.3409
Epoch 2/50
555/555 [==============================] - 2s 4ms/step - loss: 0.2590
Epoch 3/50
555/555 [==============================] - 2s 4ms/step - loss: 0.2545
Epoch 4/50
555/555 [==============================] - 2s 4ms/step - loss: 0.2513
Epoch 5/50
555/555 [==============================] - 2s 4ms/step - loss: 0.2488
Epoch 6/50
555/555 [==============================] - 2s 4ms/step - loss: 0.2465
Epoch 7/50
555/555 [==============================] - 2s 4ms/step - loss: 0.2444
Epoch 8/50
555/555 [==============================] - 2s 4ms/step - loss: 0.2427
Epoch 9/50
555/555 [==============================] - 2s 4ms/step - loss: 0.2412
Epoch 10/50
555/555 [==============================] - 2s 4ms/step - loss: 0.2396
Epoch 11/50
555/555 [==============================] - 2s 4ms/step - loss: 0.2374
Epoch 12/50
555/555 [==============================] - 2s 4ms/step - loss: 0.2366
Epoch 13/50
555/555 [====

* на 200 эпох

In [72]:
kerasModel = createKerasModel(data['X_train'].shape[1])
config = {'epochs': 200, 'model': kerasModel}
res_table, models = myPredict(res_table, data, WrapperKeras, config)

Epoch 1/200
555/555 [==============================] - 5s 3ms/step - loss: 0.3402
Epoch 2/200
555/555 [==============================] - 2s 3ms/step - loss: 0.2579
Epoch 3/200
555/555 [==============================] - 2s 3ms/step - loss: 0.2535
Epoch 4/200
555/555 [==============================] - 2s 3ms/step - loss: 0.2506
Epoch 5/200
555/555 [==============================] - 2s 3ms/step - loss: 0.2484
Epoch 6/200
555/555 [==============================] - 2s 3ms/step - loss: 0.2461
Epoch 7/200
555/555 [==============================] - 2s 3ms/step - loss: 0.2441
Epoch 8/200
555/555 [==============================] - 2s 3ms/step - loss: 0.2421
Epoch 9/200
555/555 [==============================] - 2s 3ms/step - loss: 0.2402
Epoch 10/200
555/555 [==============================] - 2s 3ms/step - loss: 0.2384
Epoch 11/200
555/555 [==============================] - 2s 3ms/step - loss: 0.2374
Epoch 12/200
555/555 [==============================] - 2s 3ms/step - loss: 0.2358
Epoch 13/200


555/555 [==============================] - 2s 4ms/step - loss: 0.2191
Epoch 100/200
555/555 [==============================] - 2s 4ms/step - loss: 0.2189
Epoch 101/200
555/555 [==============================] - 2s 4ms/step - loss: 0.2190
Epoch 102/200
555/555 [==============================] - 2s 4ms/step - loss: 0.2188
Epoch 103/200
555/555 [==============================] - 2s 3ms/step - loss: 0.2192
Epoch 104/200
555/555 [==============================] - 2s 4ms/step - loss: 0.2191
Epoch 105/200
555/555 [==============================] - 2s 4ms/step - loss: 0.2189
Epoch 106/200
555/555 [==============================] - 2s 4ms/step - loss: 0.2191
Epoch 107/200
555/555 [==============================] - 2s 4ms/step - loss: 0.2189
Epoch 108/200
555/555 [==============================] - 2s 4ms/step - loss: 0.2188
Epoch 109/200
555/555 [==============================] - 2s 3ms/step - loss: 0.2191
Epoch 110/200
555/555 [==============================] - 2s 3ms/step - loss: 0.2190
Epoch 

555/555 [==============================] - 2s 4ms/step - loss: 0.2159
Epoch 197/200
555/555 [==============================] - 2s 4ms/step - loss: 0.2161
Epoch 198/200
555/555 [==============================] - 2s 4ms/step - loss: 0.2157
Epoch 199/200
555/555 [==============================] - 2s 4ms/step - loss: 0.2161
Epoch 200/200
2220/2220 [==============================] - 2s 783us/step
name: WrapperKeras: {"epochs": 200}
train AUC: 0.8691353109411597
test AUC: 0.8318700527057322
train ACC: 0.9215771840423436
test ACC: 0.9184661730341508
config: {'epochs': 200}


In [54]:
res_table

,name,train AUC,test AUC,train ACC,test ACC,config
0,"WrapperNaive: {""binarize"": 2}",0.612746,0.614147,0.891287,0.892297,{'binarize': 2}
1,"WrapperNaive: {""binarize"": 5}",0.543933,0.538692,0.912547,0.913610,{'binarize': 5}
2,"WrapperKeras: {""epochs"": 50}",0.850541,0.829366,0.919430,0.920141,{'epochs': 50}
3,"WrapperKeras: {""epochs"": 200}",0.867842,0.833780,0.920785,0.920254,{'epochs': 200}
4,"WrapperMLPGrid: {""solver"": [""adam""], ""max_iter...",0.847155,0.800624,0.919272,0.919423,"{'solver': ['adam'], 'max_iter': [50, 100], 'a..."
5,"WrapperRFGridSearch: {""n_estimators"": [200], ""...",0.999946,0.841568,0.993243,0.920747,"{'n_estimators': [200], 'max_features': ['sqrt..."
6,"WrapperRFGridSearch: {""n_estimators"": [500], ""...",0.999953,0.843758,0.993479,0.920817,"{'n_estimators': [500], 'max_features': ['sqrt..."
7,"WrapperKeras: {""epochs"": 200}",0.866766,0.832867,0.921180,0.920367,{'epochs': 200}


#### Запускаем Multi-layer Perceptron classifier

In [30]:
%%time
param_grid = {
    'solver': ['adam'],
    'max_iter': [50, 100], #  
    'activation': ['relu', 'logistic'], # 
    'hidden_layer_sizes': [(62, 32, 1)]
}

res_table, models = myPredict(res_table, data, WrapperMLPGrid, param_grid)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


/usr/local/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py

name: WrapperMLPGrid: {"solver": ["adam"], "max_iter": [50, 100], "activation": ["relu", "logistic"], "hidden_layer_sizes": [[62, 32, 1]]}
train AUC: 0.8471553048862996
test AUC: 0.8006244194482505
train ACC: 0.9192720797319744
test ACC: 0.9194234071905177
config: {'solver': ['adam'], 'max_iter': [50, 100], 'activation': ['relu', 'logistic'], 'hidden_layer_sizes': [(62, 32, 1)]}
CPU times: user 8min 34s, sys: 1min 18s, total: 9min 53s
Wall time: 13min 8s


In [31]:
res_table

,name,train AUC,test AUC,train ACC,test ACC,config
0,"WrapperNaive: {""binarize"": 2}",0.612746,0.614147,0.891287,0.892297,{'binarize': 2}
1,"WrapperNaive: {""binarize"": 5}",0.543933,0.538692,0.912547,0.913610,{'binarize': 5}
2,"WrapperKeras: {""epochs"": 50}",0.850541,0.829366,0.919430,0.920141,{'epochs': 50}
3,"WrapperKeras: {""epochs"": 200}",0.867842,0.833780,0.920785,0.920254,{'epochs': 200}
4,"WrapperMLPGrid: {""solver"": [""adam""], ""max_iter...",0.847155,0.800624,0.919272,0.919423,"{'solver': ['adam'], 'max_iter': [50, 100], 'a..."


#### Запускаем RandomForestClassifier

In [32]:
%%time

param_grid = {
    'n_estimators': [200], # количество деревьев в лесу
    'max_features': ['sqrt'], #  размер случайных подмножеств функций, которые следует учитывать при разделении узла
    'max_depth': [32], # максимальная глубина дерева
    'criterion': ['entropy'], # функция измерения качества разделения
}

res_table, models = myPredict(res_table, data, WrapperRFGridSearch, param_grid)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
name: WrapperRFGridSearch: {"n_estimators": [200], "max_features": ["sqrt"], "max_depth": [32], "criterion": ["entropy"]}
train AUC: 0.999945718316805
test AUC: 0.8415676182617302
train ACC: 0.9932430530138799
test ACC: 0.9207466426419663
config: {'n_estimators': [200], 'max_features': ['sqrt'], 'max_depth': [32], 'criterion': ['entropy']}
CPU times: user 4min 11s, sys: 1.29 s, total: 4min 12s
Wall time: 8min 34s


In [42]:
res_table

,name,train AUC,test AUC,train ACC,test ACC,config
0,"WrapperNaive: {""binarize"": 2}",0.612746,0.614147,0.891287,0.892297,{'binarize': 2}
1,"WrapperNaive: {""binarize"": 5}",0.543933,0.538692,0.912547,0.913610,{'binarize': 5}
2,"WrapperKeras: {""epochs"": 50}",0.850541,0.829366,0.919430,0.920141,{'epochs': 50}
3,"WrapperKeras: {""epochs"": 200}",0.867842,0.833780,0.920785,0.920254,{'epochs': 200}
4,"WrapperMLPGrid: {""solver"": [""adam""], ""max_iter...",0.847155,0.800624,0.919272,0.919423,"{'solver': ['adam'], 'max_iter': [50, 100], 'a..."
5,"WrapperRFGridSearch: {""n_estimators"": [200], ""...",0.999946,0.841568,0.993243,0.920747,"{'n_estimators': [200], 'max_features': ['sqrt..."
6,"WrapperRFGridSearch: {""n_estimators"": [500], ""...",0.999953,0.843758,0.993479,0.920817,"{'n_estimators': [500], 'max_features': ['sqrt..."


### Делаем микс моделей чтобы достичь `0.85`

In [43]:
def compositionModel(composition_models, weights):
    pred = 0
    for i in range(len(composition_models)):
        model = composition_models[i]
        weight = weights[i]
        print(model, pred)
        pred += model.predict(data['X_test']) * weight
    print(roc_auc_score(data['y_test'], pred / len(composition_models)))

In [44]:
models

{'WrapperNaive: {"binarize": 2}': <__main__.WrapperNaive at 0x165324130>,
 'WrapperNaive: {"binarize": 5}': <__main__.WrapperNaive at 0x1653249a0>,
 'WrapperKeras: {"epochs": 50}': <__main__.WrapperKeras at 0x16551ff10>,
 'WrapperKeras: {"epochs": 200}': <__main__.WrapperKeras at 0x107ca37c0>,
 'WrapperMLPGrid: {"solver": ["adam"], "max_iter": [50, 100], "activation": ["relu", "logistic"], "hidden_layer_sizes": [[62, 32, 1]]}': <__main__.WrapperMLPGrid at 0x166bbd600>,
 'WrapperRFGridSearch: {"n_estimators": [200], "max_features": ["sqrt"], "max_depth": [32], "criterion": ["entropy"]}': <__main__.WrapperRFGridSearch at 0x166bda1d0>,
 'WrapperRFGridSearch: {"n_estimators": [500], "max_features": ["sqrt"], "max_depth": [32], "criterion": ["entropy"]}': <__main__.WrapperRFGridSearch at 0x16676db10>}

In [55]:
keras = models['WrapperKeras: {"epochs": 200}']
rf200 = models['WrapperRFGridSearch: {"n_estimators": [200], "max_features": ["sqrt"], "max_depth": [32], "criterion": ["entropy"]}']

In [59]:
compositionModel([keras, rf200], [0.95, 1])

<__main__.WrapperKeras object at 0x16751b940> 0
2220/2220 [==============================] - 2s 1ms/step
<__main__.WrapperRFGridSearch object at 0x16676db10> [0.04557562 0.22155613 0.03257994 ... 0.1842497  0.19787842 0.28719074]
0.8478549319477848


In [45]:
models_model = {}
for key in models:
    if('MLPClassifier' not in key):
        models_model[key] = models[key].model

## Сохранение результатов

In [46]:
joblib.dump(models_model, 'save/models_ai-small-preproc.pkl')
joblib.dump(data, 'save/data_ai-small-preproc.pkl')

Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers
......batch_normalization
.........vars
............0
............1
............2
............3
......batch_normalization_1
.........vars
............0
............1
............2
............3
......batch_normalization_2
.........vars
............0
............1
............2
............3
......batch_normalization_3
.........vars
............0
............1
............2
............3
......batch_normalization_4
.........vars
............0
............1
............2
............3
......dense
.........vars
............0
............1
......dense_1
.........vars
............0
............1
......dense_2
.........vars
............0
............1
......dense_3
.........vars
............0
............1
......dense_4
.........vars
............0
............1
......dense_5
.........vars
............0
............1
......dropout
.........vars
...metrics
......mean
.........vars
............0
............1
...optimiz

['save/data_ai-small-preproc.pkl']

In [40]:
res_table

,name,train AUC,test AUC,train ACC,test ACC,config
0,"WrapperNaive: {""binarize"": 2}",0.612746,0.614147,0.891287,0.892297,{'binarize': 2}
1,"WrapperNaive: {""binarize"": 5}",0.543933,0.538692,0.912547,0.913610,{'binarize': 5}
2,"WrapperKeras: {""epochs"": 50}",0.850541,0.829366,0.919430,0.920141,{'epochs': 50}
3,"WrapperKeras: {""epochs"": 200}",0.867842,0.833780,0.920785,0.920254,{'epochs': 200}
4,"WrapperMLPGrid: {""solver"": [""adam""], ""max_iter...",0.847155,0.800624,0.919272,0.919423,"{'solver': ['adam'], 'max_iter': [50, 100], 'a..."
5,"WrapperRFGridSearch: {""n_estimators"": [200], ""...",0.999946,0.841568,0.993243,0.920747,"{'n_estimators': [200], 'max_features': ['sqrt..."
